In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import molsysmt as msm

In [3]:
molsys = msm.convert('1HKC')

/home/diego/projects@dprada/MolSysMT/molsysmt/item/mmtf_MMTFDecoder/to_molsysmt_Topology.py:32: UserWarning: The structure in the PDB has biological assemblies. There are geometrical transformations proposed in the structure. See the following issue in the source code repository: https://github.com/uibcdf/MolSysMT/issues/33
  warnings.warn(warning_message)
/home/diego/projects@dprada/MolSysMT/molsysmt/item/mmtf_MMTFDecoder/to_molsysmt_Topology.py:42: UserWarning: The structure in the PDB has biological assemblies. There are geometrical transformations proposed in the structure. See the following issue in the source code repository: https://github.com/uibcdf/MolSysMT/issues/33
  warnings.warn(warning_message)


The structure in the PDB has biological assemblies. There are geometrical transformations proposed in the structure. See the following issue in the source code repository: https://github.com/uibcdf/MolSysMT/issues/33


In [4]:
mmtf = msm.convert('1HKC', to_form='mmtf.MMTFDecoder')

In [14]:
mmtf.bio_assembly[0]

{'transformList': [{'chainIndexList': [0, 1, 2, 3, 4, 5, 6],
   'matrix': [1.0,
    0.0,
    0.0,
    0.0,
    0.0,
    1.0,
    0.0,
    0.0,
    0.0,
    0.0,
    1.0,
    0.0,
    0.0,
    0.0,
    0.0,
    1.0]},
  {'chainIndexList': [0, 1, 2, 3, 4, 5, 6],
   'matrix': [1.0,
    0.0,
    0.0,
    0.0,
    0.0,
    -1.0,
    0.0,
    303.2647758972,
    0.0,
    0.0,
    -1.0,
    33.1566666667,
    0.0,
    0.0,
    0.0,
    1.0]}],
 'name': '1'}

In [9]:
mmtf.bio_assembly[0].keys()

dict_keys(['transformList', 'name'])

In [18]:
mmtf.bio_assembly[0]['transformList'][0]

{'chainIndexList': [0, 1, 2, 3, 4, 5, 6],
 'matrix': [1.0,
  0.0,
  0.0,
  0.0,
  0.0,
  1.0,
  0.0,
  0.0,
  0.0,
  0.0,
  1.0,
  0.0,
  0.0,
  0.0,
  0.0,
  1.0]}

In [26]:
transf = np.array(mmtf.bio_assembly[0]['transformList'][1]['matrix']).reshape(-1, 4)

In [27]:
transf

array([[  1.        ,   0.        ,   0.        ,   0.        ],
       [  0.        ,  -1.        ,   0.        , 303.2647759 ],
       [  0.        ,   0.        ,  -1.        ,  33.15666667],
       [  0.        ,   0.        ,   0.        ,   1.        ]])

In [28]:
rot = transf[:3,:3]

In [29]:
rot

array([[ 1.,  0.,  0.],
       [ 0., -1.,  0.],
       [ 0.,  0., -1.]])

In [30]:
trans = transf[:3,3]

In [31]:
trans

array([  0.        , 303.2647759 ,  33.15666667])

In [22]:
import numpy as np

In [ ]:
msm.get(molsys, element='atom', selection='atom_name=="CA"', b_factor=True)

In [ ]:
molsys = msm.build.remove_atoms_with_alternate_locations(molsys)

In [ ]:
msm.view(molsys)

In [ ]:
my_enm = oenm.ElasticNetworkModel('1TCD', selection='atom_name=="CA"', cutoff='12 angstroms')